## Concorde TSP  
from:   
https://www.kaggle.com/blacksix/concorde-for-5-hours  
https://www.kaggle.com/blacksix/concorde-for-5-min

In [ ]:
%%bash -e
if ! [[ -f ./linkern ]]; then
  wget http://www.math.uwaterloo.ca/tsp/concorde/downloads/codes/src/co031219.tgz
  echo 'c3650a59c8d57e0a00e81c1288b994a99c5aa03e5d96a314834c2d8f9505c724  co031219.tgz' | sha256sum -c
  tar xf co031219.tgz
  (cd concorde && CFLAGS='-Ofast -march=native -mtune=native -fPIC' ./configure)
  (cd concorde/LINKERN && make -j && cp linkern ../../)
  rm -rf concorde co031219.tgz
fi

In [139]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import sympy
from numba import jit

In [2]:
def read_cities(filename='input/cities.csv'):
    return pd.read_csv(filename, index_col=['CityId'])

cities = read_cities()

In [ ]:
cities1k = cities * 1000

In [ ]:
%%bash
time ./linkern -K 1 -s 42 -S linkern.tour -R 999999999 -t 300 ./cities1k.tsp >linkern.log

In [ ]:
!sed -Ene 's/([0-9]+) Steps.*Best: ([0-9]+).*/\1,\2/p' linkern.log >linkern.csv
pd.read_csv('linkern.csv', index_col=0, names=['TSP tour length']).plot();

In [ ]:
class Tour:
    cities = read_cities()
    coords = (cities.X + 1j * cities.Y).values
    penalized = ~cities.index.isin(sympy.primerange(0, len(cities)))

    def __init__(self, data):
        """Initializes from a list/iterable of indexes or a filename of tour in csv/tsplib/linkern format."""

        if type(data) is str:
            data = self._read(data)
        elif type(data) is not np.ndarray or data.dtype != np.int32:
            data = np.array(data, dtype=np.int32)
        self.data = data

        if (self.data[0] != 0 or self.data[-1] != 0 or len(self.data) != len(self.cities) + 1):
            raise Exception('Invalid tour')

    @classmethod
    def _read(cls, filename):
        data = open(filename, 'r').read()
        if data.startswith('Path'):  # csv
            return pd.read_csv(io.StringIO(data)).Path.values
        offs = data.find('TOUR_SECTION\n')
        if offs != -1:  # TSPLIB/LKH
            data = np.fromstring(data[offs+13:], sep='\n', dtype=np.int32)
            data[-1] = 1
            return data - 1
        else:  # linkern
            data = data.replace('\n', ' ')
            data = np.fromstring(data, sep=' ', dtype=np.int32)
            if len(data) != data[0] + 1:
                raise Exception('Unrecognized format in %s' % filename)
            return np.concatenate((data[1:], [0]))

    def info(self):
        dist = np.abs(np.diff(self.coords[self.data]))
        penalty = 0.1 * np.sum(dist[9::10] * self.penalized[self.data[9:-1:10]])
        dist = np.sum(dist)
        return { 'score': dist + penalty, 'dist': dist, 'penalty': penalty }

    def dist(self):
        return self.info()['dist']

    def score(self):
        return self.info()['score']

    def __repr__(self):
        return 'Tour: %s' % str(self.info())

    def to_csv(self, filename):
        pd.DataFrame({'Path': self.data}).to_csv(filename, index=False)

In [ ]:
tour = Tour('linkern.tour')
tour # Tour: {'score': 1518043.4728760184, 'dist': 1504346.798813631, 'penalty': 13696.674062387363}

In [ ]:
tour.to_csv('submission.csv')

In [ ]:
def plot_tour(tour, cmap=mpl.cm.gist_rainbow, figsize=(25, 20)):
    fig, ax = plt.subplots(figsize=figsize)
    n = len(tour.data)

    for i in range(201):
        ind = tour.data[n//200*i:min(n, n//200*(i+1)+1)]
        ax.plot(tour.cities.X[ind], tour.cities.Y[ind], color=cmap(i/200.0), linewidth=1)

    ax.plot(tour.cities.X[0], tour.cities.Y[0], marker='*', markersize=15, markerfacecolor='k')
    ax.autoscale(tight=True)
    mpl.colorbar.ColorbarBase(ax=fig.add_axes([0.125, 0.075, 0.775, 0.01]),
                              norm=mpl.colors.Normalize(vmin=0, vmax=n),
                              cmap=cmap, orientation='horizontal')

plot_tour(tour)

In [140]:
from sympy import isprime, primerange
df = pd.read_csv('submission.csv')
cities = pd.read_csv('input/cities.csv')

# transfar Dataframe to numpy for Numba
primes = np.array(list(primerange(0, len(cities))))
penalized = ~cities.index.isin(sympy.primerange(0, len(cities)))
complexes = np.array(cities.X + 1j * cities.Y)
df_path = np.array(df.Path)
len(df_path)

197770

In [141]:
[cities.X.min(), cities.X.max()]

[1.87192466558405, 5099.50214180651]

In [142]:
[cities.Y.min(), cities.Y.max()]

[0.0, 3397.80982412656]

In [143]:
[len(cities.X.unique()), len(cities.Y.unique())]

[197742, 197750]

In [144]:
@jit
def dist(a, b):
    d = abs(complexes[a] - complexes[b])
    return d

In [145]:
@jit
def total_score(path):
    score = 0
    score_no_penalty = 0
    no_penalty = 0
    for i in range(1, len(path)):
        d = dist(path[i-1], path[i])
        score_no_penalty += d
        if i % 10 == 0:
            if not isprime(path[i-1]):
                d = d * 1.1
            else:
                no_penalty += 1
        score += d
    return score, score_no_penalty, no_penalty

In [184]:
total_score(df_path)

(1518043.4728760002, 1504346.7988136157, 1815)

In [149]:
@jit
def score_np(a, b):
    c = []
    for i in df_path[a: b+1]:
        c.append(complexes[i])
    score = np.sum(np.abs(np.diff(c)))
    return score

In [150]:
score_np(8, 9)

6.762894404297583

In [151]:
@jit('i8(i8)')
def closest(base_index):
    min_dist = np.inf
    closest_index = -1
    for i in df_path:
        d = dist(base_index, i)
        if d < min_dist and i != base_index:
            min_dist = d
            closest_index = i
    return closest_index

In [152]:
closest(111804)

52086

In [153]:
@jit('i8(i8)')
def closest_prime(base_index):
    min_dist = np.inf
    closest_index = -1
    for i in primes:
        d = dist(base_index, i)
        if d < min_dist and i != base_index:
            min_dist = d
            closest_index = i
    return closest_index

In [154]:
closest_prime(df_path[8])

38447

In [57]:
closest(8)

20433

In [209]:
# if prime before i 
a = np.array([0,1,2,3,4,5,6,7,8,9,10,11])
prime_index = 10
target_index = 0
target_value = a[target_index]
a = np.insert(a, prime_index, target_value)
a = np.delete(a, target_index)
a

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9,  0, 10, 11])

In [224]:
# if prime after i
a = np.array([0,1,2,3,4,5,6,7,8,9,10,11,12,13])
prime_index = 9
target_index = 13
target_value = a[target_index]
b = np.insert(a, prime_index, target_value)
b = np.delete(b, target_index + 1)
b

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8, 13,  9, 10, 11, 12])

In [226]:
def score_between(path, a, b):
    path_complexes = 
    for i in 

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8, 13,  9, 10, 11, 12])

In [ ]:
def opt1(df_path):
    s = 0
    df_copy = df_path.copy()
    for i in range(0, len(df_path), 10):
        if i > 0:
            p = isprime(df_path[i-1])
            if p == False:
                cp = closest_prime(df_path[i-2])
                cp_df_index = np.where(df_path==cp)[0][0]
                if cp_df_index < i-1:
                    score_before = score(df_path, cp_df_index-1, i)
                    df_copy = np.insert(df_copy, i, cp)
                    df_copy = np.delete(df_copy, cp_df_index)
                    
                    

In [171]:
@jit
def opt(df_path):
    sum8 = 0
    sum9 = 0
    sum10 = 0
    for i in range(0, len(df_path), 10):
        if i > 0:
            p8 = isprime(df_path[i-2])
            p9 = isprime(df_path[i-1])
            p10 = isprime(df_path[i])
            if p8 == True and p9 == False and p10 == False:
                d78 = dist(df_path[i-3], df_path[i-2]) # 7-8
                d89 = dist(df_path[i-2], df_path[i-1]) # 8-9
                d910 = dist(df_path[i-1], df_path[i])  # 9-10 with penalty
                d = d78 + d89 + d910*1.1
                e79 = dist(df_path[i-3], df_path[i-1]) # 7-9
                e98 = dist(df_path[i-1], df_path[i-2]) # 9-8
                e810 = dist(df_path[i-2], df_path[i])  # 8-10 no penalty
                e = e79 + e98 + e810
                if e < d: # 5 3.5466290514118803
                    df_path[i-2], df_path[i-1] = df_path[i-1], df_path[i-2]
                    sum8 += (d - e)
            if p8 == False and p9 == False and p10 == True:
                f89 = dist(df_path[i-2], df_path[i-1])
                f910 = dist(df_path[i-1], df_path[i])
                f1011 = dist(df_path[i], df_path[i+1])
                f = f89 + f910 + f1011*1.1
                g810 = dist(df_path[i-2], df_path[i])
                g109 = dist(df_path[i], df_path[i-1])
                g911 = dist(df_path[i-1], df_path[i+1])
                g = g810 + g109 + g911
                if g < f: # 4 2.008119222389432
                    df_path[i-1], df_path[i] = df_path[i], df_path[i-1]
                    sum10 += (f - g)
            if p8 + p9 + p10 == 0:
                cp = closest_prime(df_path[i-2])
                cp_index = np.where(df_path==cp)[0][0]
                if cp_index > i:
                    a89 = dist(df_path[i-2], df_path[i-1])
                    a910 = dist(df_path[i-1], df_path[i])
                    a = a89 + a910*1.1
                    
                    b8cp = dist(df_path[i-2], cp)
                    bcp9 = dist(cp, df_path[i-1])
                    b = b8cp + bcp9
                    if b < a:
                        df_path[i-1] = 
                        sum10 += (a - b)
                
    #print(cnt8, sum8)
    print(sum8, sum9, sum10)
    
# p8: 1796, p9: 1815, p10: 1758, none: 14867

In [172]:
opt(df_path)

83 126.00341568676401


In [ ]:
np = 0
pe = 0

for i in range(0, len(df_path), 10):
    if i > 0:
        p8 = isprime(df_path[i-2])
        p9 = isprime(df_path[i-1])
        p10 = isprime(df_path[i])
        if p8+p9+p10 == 0:
            np += 1
        else:
            pe += 1
#         if p9 == False:
#             if p8 == False and p10 == False:
#                 pm.append(i)
#             elif p8 == False and p10 == True:
#                 pm8.append(i)
#             elif p8 == True and p10 == False:
#                 pm10.append(i)
#             d_8_9 = dist3(df.Path[i-2], df.Path[i-1])
#             d_9_10 = dist3(df.Path[i-1], df.Path[i]) * 1.1
#             d = d_8_9 + d_9_10
            
#             cp = closest_prime(df.Path[i])
#             dp_8_9 = dist3(df.Path[i-2], cp)
#             dp_9_10 = dist3(cp, df.Path[i-1])
#             dp = dp_8_9 + dp_9_10
            
#             if dp < d:
#                 pm.append(i)
[np, pe]

In [ ]:
len(primes)

In [ ]:
i = 130
d_8_9 = dist3(df.Path[i-2], df.Path[i-1])
d_9_10 = dist3(df.Path[i-1], df.Path[i]) * 1.1
d = d_8_9 + d_9_10
d

In [ ]:
i = 130
cp = closest_prime(df.Path[i])
dp_8_9 = dist3(df.Path[i-2], cp)
dp_9_10 = dist3(cp, df.Path[i])
dp = dp_8_9 + dp_9_10

In [ ]:
dp